<a href="https://colab.research.google.com/github/Alexher90/An-lisis-de-Datos-2/blob/master/Solucion_Semana13_FiltradoColaborativo_Surprise_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MIIA-4203 MODELOS AVANZADOS PARA ANÁLISIS DE DATOS II


# Sistemas de Recomendación de filtrado colaborativo con SurPRISE

## Actividad 13

### Profesor: Camilo Franco (c.franco31@uniandes.edu.co)

En este cuadernos estudiaremos distintos métodos de filtrado colaborativo, junto con algunas métricas de desempeño de los algoritmos de recomendación. En la actividad anterior vimos un primer método dentro de la familia de métodos de factores latentes (por factorización matricial) con preferencias implícitas. En este cuaderno vamos a explorar otra familia de métodos, los cuales exploran *vecindades* o relaciones entre items o entre usuarios, basados en los K-vecinos más cercanos. Vamos a estudiar la aplicación de estos métodos sobre preferencias explícitas de la mano de la biblioteca de Python SurPRISE (Simple Python Recommendation System Engine) http://surpriselib.com/


**Nombres: Alexander Hernández Páez (200920588), Alexander Camargo (200911325), Juan David Cortés (201728568), Wilson Felipe González (200924943).**

Vamos a utilizar los siguientes paqutes:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime

Además vamos a usar `plotly`, una bilbioteca de Python que permite visualizar los datos bajo múltiples tipos de gráficos. Además cuenta con un amplio rango de casos de uso. Este paquete plotly.py está construido sobre la bilbioteca Plotly JavaScript (plotly.js), permitiendo crear visualizaciones interactivas (para visualización en Jupyter, que se pueden guardar como archivos HTML, o que también se pueden usar en aplicaciones de Python en la web usando Dash https://plotly.com/dash/).

https://plotly.com/python/getting-started/?utm_source=mailchimp-jan-2015&utm_medium=email&utm_campaign=generalemail-jan2015&utm_term=bubble-chart

Para su instalacion en Anaconda: $ conda install -c plotly plotly=4.7.0

En GoogleColab se puede importar directamente: https://colab.research.google.com/notebooks/charts.ipynb#scrollTo=8YCVGqZkJJxT

In [ ]:
import plotly
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go

Finalmente vamos a usar SurPRISE. 

Para su instalación, debe ejecutar la siguiente linea desde la terminal (conda prompt): 

`conda install -c conda-forge scikit-surprise`

Si tiene problemas, puede chequear la version de scipy en su computador. Debe ser una version reciente.
Para ello ejecute: pip install "scipy>=1.0" 
 ..y para instalar la bilbioteca: pip install scikit-surprise
 
*Para ejecutar SurPRISE en GoogleColab,*
- !pip uninstall -y scipy
- !pip install scipy==1.0.0
- !pip install surprise

In [ ]:
import surprise

from surprise import Reader
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import LeaveOneOut

## 1. Visualización de los datos

Trabajemos con nuestros datos de peliculas IMDB (https://www.imdb.com/) con los ratings que algunos usuarios le han dado a algunas peliculas: 

In [ ]:
pelis = pd.read_csv('movies_metadata.csv', low_memory=False)

df = pd.read_csv('ratings_small.csv')
df.drop(['timestamp'], axis=1, inplace=True)

df.head()

Exploremos la distribucion de los ratings:

In [ ]:
init_notebook_mode()#(connected=True)

data = df['rating'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Creamos el diseño
layout = dict(title = 'Distribucion de los {} ratings'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Frecuencias'))
# Graficamos
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)
#plot(fig)

Veamos la distribucion de ratings por pelicula:

In [ ]:
# Conteo de ratings por pelicula
data = df.groupby('movieId')['rating'].count()

trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 100,
                                  size = 2))
# Diseño
layout = go.Layout(title = 'Distribucion del numero de ratings por pelicula (hasta 100 peliculas)',
                   xaxis = dict(title = 'Ratings por pelicula'),
                   yaxis = dict(title = 'Peliculas'),
                   bargap = 0.2)

# Grafica
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Vemos que la mayoría de peliculas reciben un solo rating! Y la que más, recibe 341 ratings:

In [ ]:
df.groupby('movieId')['rating'].count().reset_index().sort_values('rating', ascending=False)[:10]

Por ejemplo, la pelicula con ID=296 recibe 324 ratings de los usuarios:

In [ ]:
pelis[pelis.id=='296']['original_title']

Ahora veamos la distribucion de ratings por usuario:

In [ ]:
# Conteo de ratings por usuario
data = df.groupby('userId')['rating'].count().clip(upper=200)

trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 200,
                                  size = 2))
# Diseño
layout = go.Layout(title = 'Distribucion del numero de ratings por usuario (hasta 200 usuarios)',
                   xaxis = dict(title = 'Ratings por usuario'),
                   yaxis = dict(title = 'Usuarios'),
                   bargap = 0.2)

# Grafica
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Igualmente, vemos que la mayoría de usuarios dan solo unos pocos ratings!

Veamos los usuarios que dan un mayor numero de ratings:

In [ ]:
df.groupby('userId')['rating'].count().reset_index().sort_values('rating', ascending=False)[:10]

Por ejemplo, el usuario que más ha calificado películas ha calificado 2391 peliculas:

In [ ]:
df[df.userId==547].shape

Entonces observamos que muchos usuarios han calificado menos de 30 peliculas, y que muchas peliculas han recibido menos de 5 ratings. 

Con el fin de reducir el coste computacional de nuestros calculos y poder ejecutar más fácilmente el código que sigue vamos a fitrar las peliculas menos calificadas y los usuarios menos activos.

In [ ]:
min_p_ratings = 5
filter_p = df['movieId'].value_counts() > min_p_ratings
filter_p = filter_p[filter_p].index.tolist()

min_u_ratings = 30
filter_u = df['userId'].value_counts() > min_u_ratings
filter_u = filter_u[filter_u].index.tolist()

df_nuevo = df[(df['movieId'].isin(filter_p)) & (df['userId'].isin(filter_u))]
print('Los datos originales tienen tamaño:\t{}'.format(df.shape))
print('Los nuevo datos tienen tamaño:\t{}'.format(df_nuevo.shape))

## 2. Métodos de filtrado colaborativo

A continuación vamos a implementar una batería de modelos que SurPRISE tiene programados para que nosotros los apliquemos facilmente. Estos modelos nos sirven como punto de partida para intentar mejorarlos desarrollando nuestros propios modelos.

Primero debemos cargar nuestros datos. para ello, utilizamos la función `load_from_df()`, utilizando la función `Reader` y especificando la escala en la que se da el *Rating*. En este caso los ratings van del 0 al 5. 

Nuestros datos deben constar de 3 columnas, los ids de usuario, los ids de los items, y los ratings (en este orden). Cada observación contiene el rating correspondiente. 

In [ ]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df_nuevo[['userId', 'movieId', 'rating']], reader)

data

Montados en SurPRISE, podemos implementar los siguientes algoritmos (antes de entrar en la teoría ejecutemos la celda abajo pues la validación cruzada de todos los modelos toma tiempo): 

**Algoritmos base**

- NormalPredictor

Este algoritmo predice un rating aleatorio asumiendo que la muestra de entrenamiento proviene de una distribución Normal:

$$\hat r_{ui}\sim Normal(\hat \mu, \hat \sigma)  $$

donde 

$$ \hat \mu = \frac{1}{|R_{entrenamiento}|}\sum_{r_{ui} \in R_{entrenamiento}} r_{ui}  $$

$$ \hat \sigma= \sqrt{\sum_{r_{ui} \in R_{entrenamiento}} \frac{(r_{ui}-\hat \mu)^2}{|R_{entrenamiento}|}} $$


- BaselineOnly

El algoritmo obtiene su estimación a partir del rating medio y las desviaciones observadas para la pelicula $i$ y el usuario $u$:


$$\hat r_{ui}= \mu + b_i + b_u$$

donde $\mu$ es el rating promedio de los datos de entrenamiento, $b_i$ que es el rating promedio del item $i$ menos $\mu$ y $b_u$ que es el rating promedio del usuario $u$ menos $\mu$.


**Algoritmos de vecindades:**

- KNNBasic

El *KNNBasic* es un modelo que estima los ratings de acuerdo con los $k$ vecinos más cercanos, ya sea por usuario o por item, de acuerdo con:

$$ \hat r_{ui}=\frac{\sum_{v\in N_i^k(u)}sim(u,v) \cdot r_{vi}}{\sum_{v\in N_i^k(u)}sim(u,v)}  $$

ó

$$ \hat r_{ui}=\frac{\sum_{j\in N_u^k(i)}sim(i,j) \cdot r_{uj}}{\sum_{j\in N_u^k(i)}sim(i,j)}  $$

donde $sim$ es la función de similitud (https://surprise.readthedocs.io/en/stable/similarities.html#module-surprise.similarities)


- KNNWithMeans

Este algoritmo modifica el *KNNBasic* tomando además en cuenta los ratings promedios de los usuarios:

$$ \hat r_{ui}=\mu_u + \frac{\sum_{v\in N_i^k(u)}sim(u,v) \cdot (r_{vi}-\mu_v)}{\sum_{v\in N_i^k(u)}sim(u,v)}  $$

ó

$$ \hat r_{ui}=\mu_i + \frac{\sum_{j\in N_u^k(i)}sim(i,j) \cdot (r_{uj}-\mu_j)}{\sum_{j\in N_u^k(i)}sim(i,j)}  $$


- KNNWithZScore

Este algoritmo toma en cuenta las similitudes y los ratings estandarizados:

$$ \hat r_{ui}=\mu_u + \sigma_u\frac{\sum_{v\in N_i^k(u)}sim(u,v) \cdot (r_{vi}-\mu_v)/\sigma_v}{\sum_{v\in N_i^k(u)}sim(u,v)}  $$

ó

$$ \hat r_{ui}=\mu_i + \sigma_i \frac{\sum_{j\in N_u^k(i)}sim(i,j) \cdot (r_{uj}-\mu_j)/\sigma_j}{\sum_{j\in N_u^k(i)}sim(i,j)}  $$



- KNNBaseline

Este algoritmo toma en cuenta el rating medio y las desviaciones observadas para la pelicula $i$ y el usuario $u$:

$$ \hat r_{ui}=b_{ui} + \frac{\sum_{v\in N_i^k(u)}sim(u,v) \cdot (r_{vi}-b_{vi})}{\sum_{v\in N_i^k(u)}sim(u,v)}  $$

ó

$$ \hat r_{ui}=b_{ui} + \frac{\sum_{j\in N_u^k(i)}sim(i,j) \cdot (r_{uj}-b_{uj})}{\sum_{j\in N_u^k(i)}sim(i,j)}  $$

**Algoritmos de factores latentes:**

- SVD

Este algoritmo corresponde con la factorización de la matriz de ratings (visto en la actividad anterior):

$$\hat r_{ui}=q_{i}'p_{u} + \mu + b_i + b_u$$

- SVDpp

Este algoritmo extiende el SVD al tomar en cuenta los ratings implícitos, ó la cantidad de *feedback* implícito:

$$\hat r_{ui}=\mu + b_i + b_u + q_{i}'\biggl(p_{u} + |I_u|^{-1/2} \sum_{j\in I_u} y_j \biggr) $$

donde $y_j$ es un valor binario que captura el hecho de que el usuario $u$ haya calificado o revelado su rating para $j$ (sin importar el valor del rating).

- NMF

Este algoritmo corresponde con la factorización no-negativa de la matriz de ratings, y sigue la misma formulación del SVD. Solo que se garantiza que los factores sean no-negativos.


**Algoritmo de clustering:**

- Co-clustering

En este algoritmo, los usuarios y los items son asignados a los clusters $C_u$, $C_i$ y $C_{ui}$:


$$ \hat r_{ui}=\bar{C_{ui}} + (\mu_u - \bar{C_u}) + (\mu_i - \bar{C_i})   $$

donde $\bar{C_{ui}}, \bar{C_u}, \bar{C_i}$ son respectivamente los rating promedio de los clusters $C_{ui}, C_u$ y $C_i$.

Los clusters se asignan de acuerdo con K-medias.



Por defecto se utiliza el RMSE como la métrica de error a minimizar para la predicción.

In [ ]:
benchmark = []
# Implementamos validacion cruzada sobre todos los algoritmos
for algoritmo in [SVD(), SVDpp(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    
    print("\nAlgoritmo: ", algoritmo)
    tiempo = datetime.datetime.now()
    print('\nInicia la validacion cruzada: ', tiempo)
    
    results = cross_validate(algoritmo, data, measures=['RMSE'], cv=3, verbose=False)
    
    tiempo = datetime.datetime.now()
    print('\nTermina: ', tiempo)
    
    # Guardamos los resultados
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algoritmo).split(' ')[0].split('.')[-1]], index=['Algoritmo']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algoritmo').sort_values('test_rmse')  

### Pregunta 1

Cuál método nos da mejores resultados?

**De acuerdo con el resultado del código anterior, podemos concluir que el método con los mejores resultados fue el Algoritmo de Factor Latente SVDpp, ya que obtuvo el menor RMSE calculado. Adicionalmente fue el método que más tiempo tomó.**

Enfoquémonos en los modelos que nos dan mejores resultados para entrenar y predecir los ratings de los usuarios. 

Para su implementación más eficiente, utilizamos la técnica de los Mínimos Cuadrados Alternantes (ALS), en lugar de SGD.

### Factorización matricial &  Mínimos Cuadrados Alternantes (ALS) 

Hasta ahora nos hemos dado cuenta que en Filtrado Colaborativo, la factorización matricial es una técnica clave que permite descomponer la matriz entre usuarios-items en un par de matrices rectangulares de menor dimension.

Entonces, una matriz puede ser vista como la matriz de los usuarios, donde cada fila es un usuario y cada columna es una factor latente, y la otra matriz es la de los items, donde cada fila es una factor latente y las columnas representan items. De esta manera, peliculas poco concocidas pueden obtener representaciones latentes enriquecidas, tanto como las películas más conocidas,  lo cual incrementa la capacidad predictiva de los sitemas.

En la matriz usuario-item, que es de tipo *sparse* (con muchos ceros en sus entradas), la estimacion de $r_{ui}$  se obtiene como:

$$\hat r_{ui} = \sum_{f=0}^{|F|} H_{u,f}W_{f,i}$$

donde $H$ es la matriz de usuario y $W$ es la matriz de items. 

Los factores latentes son las nuevas variables o patrones en un espacio de menores dimensiones al de la matriz original (con las observaciones sobre las interacciones usuario-item). 

El problema de la factorización matricial se puede entender como la minimización de la diferencia entre el rating observado y su estimación:

$$ argmin_{H,W} ||R-\hat R||_F + \alpha||H|| + \beta ||W||$$

que se puede solucionar por métodos de optimización como el descenso en la dirección del gradiente. Sin embargo, a medida que tenemos un mayor volumen de datos (terabytes y petabytes de datos) necesitamos escalar los algoritmos a procesos en paralelo (paradigma del Big Data).

Definamos las opciones de *baselines* para los 3 métodos (https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#baseline-estimates-configuration):

In [ ]:
bsl_options = {'method': 'als',
               'n_epochs': 20,
               'reg_u': 12, 
               'reg_i': 5  
               }

sim_options = {'name': 'cosine',
               'user_based': False  # calcula similitudes entre items
               }

Utilizamos `train_test_split()` para hacer nuestras particiones de entrenamiento y prueba y usamos la métrica de RMSE. Luego ajustamos el modelo con `fit()` sobre el entrenamiento y lo probamos con `test()` para obtener el desempeño en predicción.

La próxima celda tarda unos pocos minutos en correr:

In [ ]:
trainset, testset = train_test_split(data, test_size=0.3)

SVD = SVDpp(random_state=0)
KNN = KNNBaseline(bsl_options=bsl_options, sim_options=sim_options)
Base = BaselineOnly(bsl_options=bsl_options)

print("\nSVDpp: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predSVD = SVD.fit(trainset).test(testset)
print("RMSE del SVDpp: ", accuracy.rmse(predSVD))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)

print("\nKNNBaseline: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predKNN = KNN.fit(trainset).test(testset)
print("\nRMSE del KNNBaseline: ", accuracy.rmse(predKNN))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)

print("\nBaselineOnly: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predBase = Base.fit(trainset).test(testset)
print("\nRMSE del BaselineOnly: ", accuracy.rmse(predBase))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)


Inspeccionemos las predicciones, para ello utilicemos las siguientes funciones:

In [ ]:
def items(uid):
    """ 
    Input:
    uid: el id del usuario
    Output: 
    items calificados por el usuario
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # no se encuentra el usuario
        return 0
    
def usuarios(iid):
    """ 
    Input: 
    iid: el id del item
    Output:
    usuarios que han calificado el item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df = pd.DataFrame(predKNN, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Num_Items'] = df.uid.apply(items)
df['Num_Usuarios'] = df.iid.apply(usuarios)
df['Error'] = abs(df.est - df.rui)

mejores = df.sort_values(by='Error')[:10]
peores = df.sort_values(by='Error')[-10:]

In [ ]:
mejores[['Num_Items','Num_Usuarios','uid','iid','rui','est','Error']]

In [ ]:
print("Se tiene una pelicula con muchas calificaciones: ", 
      np.squeeze(mejores[mejores.iid == 318]['Num_Usuarios']),
      np.squeeze(pelis[pelis.id == '318']['original_title']))

print("O una pelicula con menos calificaciones: ", 
      np.squeeze(mejores[mejores.iid == 608]['Num_Usuarios']),
      np.squeeze(pelis[pelis.id == '608']['original_title']))


Ahora veamos las peores predicciones:

In [ ]:
peores[['Num_Items','Num_Usuarios','uid','iid','rui','est','Error']]

In [ ]:
print("También se tiene una pelicula con muchas calificaciones: ", 
      np.squeeze(peores[peores.iid == 2959]['Num_Usuarios']),
      np.squeeze(pelis[pelis.id == '2959']['original_title']))

print("O una pelicula con menos calificaciones: ", 
      np.squeeze(peores[peores.iid == 3160]['Num_Usuarios']),
      np.squeeze(pelis[pelis.id == '3160']['original_title']))


Examinando las peores predicciones, los errores parecen bastante altos. Veamos la ultima pelicula "Furankenshutain Tai Chitei Kaijū Baragon", que fue calificada por 34 usuarios, donde el usuario "83" calificó 98 películas.

Nuestro algoritmo del KNNBaseline predijo que este usuario calificaría esta peliucla con un 4.22, oero el usuario le dió un 0.5!

In [ ]:
#import matplotlib.pyplot as plt
#%matplotlib notebook

df_nuevo.loc[df_nuevo['movieId'] == 2959]['rating'].hist()
plt.xlabel('rating')
plt.ylabel('Numero de ratings')
plt.title('Numero de ratings de License to Wed')
plt.show()

Vemos que esta pelicula recibió mayormente ratings cercanos a 4.0, donde la mayoría de los usuarios de nuestros datos de entrenamiento calificaron la pelicula como buena en lugar de darle una mala calificación. 

Al tratarse de recomendaciones personalizadas, puede que las peores recomendaciones contengan los usuarios más raros o con gustos más peculiares.

## 3. Evaluación de sistemas y métricas

Hasta ahora hemos utilizado el RMSE para evaluar nuestros modelos, pero también podemos ver qué ocurre si usamos una medida distinta, como por ejemplo el índice FCP (*Fraction Concordant Pairs*), propuesto por (Koren & Sill 2011). Esta métrica no se concentra en el rating y su valor cardinal, sino más bien en su valor ordinal, de tal manera que captura cuándo un item es asignado una posición en el ranking de preferencia al menos tan buena como la que le asignó el usuario.

EL FCP está definido por 

$$ FCP=\frac{n_c}{n_c+n_d}$$

donde el número de items concordantes $n_c=\sum_u n_c^u$ se calcula a partir de 

$$ n_c^u = \{(i,j)|\hat r_{ui} > \hat r_{uj} \text{ y } r_{ui}>r_{uj}\} $$

y el número de items discordantes $n_d^u$ para el usuario $u$ se calcular de manera similar.

De hecho esta medida geenraliza el AUC para respuestas ordinales categóricas.

### Ejercicio 2

Encuentre la metrica FCP de los modelos previamente entrenados.

In [ ]:
trainset, testset = train_test_split(data, test_size=0.3)

SVD = SVDpp(random_state=0)
KNN = KNNBaseline(bsl_options=bsl_options, sim_options=sim_options)
Base = BaselineOnly(bsl_options=bsl_options)

print("\nSVDpp: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predSVD = SVD.fit(trainset).test(testset)
print("nFCP del SVDpp: ", accuracy.fcp(predSVD))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)

print("\nKNNBaseline: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predKNN = KNN.fit(trainset).test(testset)
print("\nFCP del KNNBaseline: ", accuracy.fcp(predKNN))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)

print("\nBaselineOnly: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predBase = Base.fit(trainset).test(testset)
print("\nFCP del BaselineOnly: ", accuracy.fcp(predBase))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)


### Pregunta 3

Cuál modelo elegiríamos basados en la FCP?

**Teniendo en cuenta que entre más alto el índice de FCP, mejor, podemos concluir que el método Base Line Only es el de mejor rendimiento.**

## Ejercicio 4

Intente mejorar los resultados del algoritmo SVD optimizando la métrica del FCP sobre los términos de regularización o el método de optimizacion (SGD o ALS).

*Ayuda: puede explorar https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.NMF*

In [ ]:
trainset, testset = train_test_split(data, test_size=0.3)

SVD = SVDpp(random_state=0, reg_bu = 12, reg_bi = 5)

print("\nSVDpp: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predSVD = SVD.fit(trainset).test(testset)
print("RMSE del SVDpp: ", accuracy.fcp(predSVD))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.3)
reg_bu = [10,11,12,13,14]
reg_bi = [3,4,5,6,7]

for i in reg_bu:
  for j in reg_bi:
    SVD = SVDpp(random_state=0, reg_bu = i, reg_bi = j)

    print("\nSVDpp: ")
    tiempo = datetime.datetime.now()
    print('\nInicia el entrenamiento y prueba: ', tiempo)

    predSVD = SVD.fit(trainset).test(testset)
    print("RMSE del SVDpp: ", accuracy.fcp(predSVD))

    tiempo = datetime.datetime.now()
    print('\nTermina: ', tiempo)

**Para el desarrollo de este ejercicio, se generó una serie de iteraciones en las que se tomaron 5 números para la regularización de los términos Bu y Bi. Luego de correr estas iteraciones, vemos que el término de regularización para Bu no genera un cambio significativo en la FCP a la medida que este se va incrementando, mientras que en caso del regularizador para Bi, podemos ver que el número mayor de la serie implementada, que fue 7, fue el que tuvo un mejor rendimiento de la FCP con un valor de 0.682.**

## Ejercicio 5

De manera análoga al ejercicio anterior, intente mejorar los resultados del algoritmo de KNN optimizando la métrica del FCP sobre el numero de vecinos o la medida de similitud utilizada.

*Ayuda: puede explorar https://surprise.readthedocs.io/en/stable/knn_inspired.html*

In [ ]:
bsl_options1 = {'method': 'als',
               'n_epochs': 20,
               'reg_u': 12, 
               'reg_i': 5  
               }

sim_options1 = {'name': 'pearson',
               'user_based': False  # calcula similitudes entre items
               }

In [ ]:
trainset, testset = train_test_split(data, test_size=0.3)

KNN = KNNBaseline(bsl_options=bsl_options1, sim_options=sim_options1)
Base = BaselineOnly(bsl_options=bsl_options1)

print("\nKNNBaseline: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predKNN = KNN.fit(trainset).test(testset)
print("\nFCP del KNNBaseline: ", accuracy.fcp(predKNN))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)

In [ ]:
bsl_options2 = {'method': 'als',
               'n_epochs': 20,
               'reg_u': 12, 
               'reg_i': 5  
               }

sim_options2 = {'name': 'msd',
               'user_based': False  # calcula similitudes entre items
               }

In [ ]:
trainset, testset = train_test_split(data, test_size=0.3)

KNN = KNNBaseline(bsl_options=bsl_options2, sim_options=sim_options2)
Base = BaselineOnly(bsl_options=bsl_options2)

print("\nKNNBaseline: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predKNN = KNN.fit(trainset).test(testset)
print("\nFCP del KNNBaseline: ", accuracy.fcp(predKNN))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)

In [ ]:
bsl_options3 = {'method': 'als',
               'n_epochs': 20,
               'reg_u': 12, 
               'reg_i': 5  
               }

sim_options3 = {'name': 'pearson_baseline',
               'user_based': False  # calcula similitudes entre items
               }

In [ ]:
trainset, testset = train_test_split(data, test_size=0.3)

KNN = KNNBaseline(bsl_options=bsl_options3, sim_options=sim_options3)
Base = BaselineOnly(bsl_options=bsl_options3)

print("\nKNNBaseline: ")
tiempo = datetime.datetime.now()
print('\nInicia el entrenamiento y prueba: ', tiempo)

predKNN = KNN.fit(trainset).test(testset)
print("\nFCP del KNNBaseline: ", accuracy.fcp(predKNN))

tiempo = datetime.datetime.now()
print('\nTermina: ', tiempo)

**Para el desarrollo de este ejercicio, se implementó el método ALS con 20 épocas. El parámetro de regularización de usuario fue de 12 y el parámetro de regularización del ítem fue de 5. Manteniendo las condiciones anteriores, y variando la medida de similitud (Cosine, Pearson, MSD y Pearson Baseline), se encontró que la medida de similitud que arroja el mayor FCP fue la de Pearson Baseline, con un valor de 0.705.**